In [1]:
import pandas as pd
import glob
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
psm_files = glob.glob("MSFragger_Output/*_psm.tsv")
print(len(psm_files))
psm_files

19


['MSFragger_Output\\acidovorans_psm.tsv',
 'MSFragger_Output\\aurantiaca_psm.tsv',
 'MSFragger_Output\\baltica_psm.tsv',
 'MSFragger_Output\\cereus_psm.tsv',
 'MSFragger_Output\\comes_psm.tsv',
 'MSFragger_Output\\cryptum_psm.tsv',
 'MSFragger_Output\\denitrificans_psm.tsv',
 'MSFragger_Output\\fragilis_psm.tsv',
 'MSFragger_Output\\gilvus_psm.tsv',
 'MSFragger_Output\\gnavus_psm.tsv',
 'MSFragger_Output\\griseorubens_psm.tsv',
 'MSFragger_Output\\indologenes_psm.tsv',
 'MSFragger_Output\\necator_psm.tsv',
 'MSFragger_Output\\palustris_psm.tsv',
 'MSFragger_Output\\smegmatis_psm.tsv',
 'MSFragger_Output\\subtilis168_psm.tsv',
 'MSFragger_Output\\thermosulf_psm.tsv',
 'MSFragger_Output\\thet_psm.tsv',
 'MSFragger_Output\\tumefaciens_psm.tsv']

In [4]:
run_files = glob.glob("MSFragger_Output/*_run.tsv")
print(len(run_files))
run_files

92


['MSFragger_Output\\acidovorans_01_run.tsv',
 'MSFragger_Output\\acidovorans_02_run.tsv',
 'MSFragger_Output\\acidovorans_03_run.tsv',
 'MSFragger_Output\\aurantiaca_01_run.tsv',
 'MSFragger_Output\\aurantiaca_02_run.tsv',
 'MSFragger_Output\\aurantiaca_03_run.tsv',
 'MSFragger_Output\\baltica_01C_run.tsv',
 'MSFragger_Output\\baltica_01Inf_run.tsv',
 'MSFragger_Output\\baltica_02C_run.tsv',
 'MSFragger_Output\\baltica_02Inf_run.tsv',
 'MSFragger_Output\\baltica_03C_run.tsv',
 'MSFragger_Output\\baltica_03Inf_run.tsv',
 'MSFragger_Output\\cereus_01_run.tsv',
 'MSFragger_Output\\cereus_02_run.tsv',
 'MSFragger_Output\\cereus_03_run.tsv',
 'MSFragger_Output\\comes_Carb_run.tsv',
 'MSFragger_Output\\comes_Glc_run.tsv',
 'MSFragger_Output\\comes_LIB_run.tsv',
 'MSFragger_Output\\cryptum_01_run.tsv',
 'MSFragger_Output\\cryptum_02_run.tsv',
 'MSFragger_Output\\cryptum_03_run.tsv',
 'MSFragger_Output\\denitrificans_01_run.tsv',
 'MSFragger_Output\\denitrificans_02_run.tsv',
 'MSFragger_Outpu

In [5]:
def extract_scannum(Spectrum):
    pattern = r'\.\d*\.'
    lst = []
    for s in Spectrum:
        m = re.search(pattern, s)
        if m:
            m = m[0]
            m = m.replace('.', '')
            m = int(m)
            lst.append(m)
        else:
            lst.append(None)
    return (lst)

In [6]:
def combine_run_files(files):
    dfs = []
    for file in files:
        name = re.split('_run.tsv', file)
        name = name[0]
        pattern = r"\\"
        name = re.split(pattern, name)
        name = name[1]
        name = re.split('_', name)
        name = name[0]
        df = pd.read_csv(file, sep = '\t')
        keeps = ['scannum', 'best_locs', 'peptide']
        df = df[df.best_locs.notnull()]
        df = df[keeps]
        df['Organism'] = [name] * len(df)
        df = df.rename(columns = {'peptide': 'Peptide'})
        dfs.append(df)
    dfs = pd.concat(dfs)
    return (dfs)


In [7]:
def combine_psm_files(files):
    dfs = []
    i = 0
    for file in files:
        name = re.split('_psm.tsv', file)
        name = name[0]
        pattern = r"\\"
        name = re.split(pattern, name)
        name = name[1]
        keeps = ['scannum', 'Peptide', 'Observed Modifications', 'Protein ID', 'Gene']
        df = pd.read_csv(file, sep = '\t')   
        df = df[df['PeptideProphet Probability'] >=0.99]
        df = df[df['Observed Modifications'].notnull()]
        df = df[df.Gene.notnull()]
        df = df[~df.Gene.str.contains('Daci')]
        df['scannum'] = extract_scannum(df.Spectrum)
        df = df[keeps]
        print(name)
        df['Organism'] = [name] * len(df)        
        dfs.append(df)
    df = pd.concat(dfs)
    return(df)

In [8]:
combinded_psm_df = combine_psm_files(psm_files)
combined_run_df = combine_run_files(run_files)

acidovorans
aurantiaca
baltica


C:\Users\nanel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


cereus
comes
cryptum
denitrificans
fragilis
gilvus
gnavus
griseorubens
indologenes
necator
palustris
smegmatis
subtilis168
thermosulf
thet
tumefaciens


C:\Users\nanel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [27]:
len(pd.unique(combined_run_df.Organism))

20

In [26]:
len(pd.unique(combinded_psm_df.Organism))

19

In [30]:
pd.unique(combined_run_df[~combined_run_df.Organism.isin(combinded_psm_df.Organism)].Organism)

array(['freundii'], dtype=object)

In [11]:
df = pd.merge(combinded_psm_df, combined_run_df, how='inner')
df

,scannum,Peptide,Observed Modifications,Protein ID,Gene,Organism,best_locs
0,1982,AAASSTAAAKTPAKR,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",A9BMX9,rpoD,acidovorans,AAASSTAAAkTPAKR
1,2020,AAASSTAAAKTPAKR,"Ala->Val(28.031300), Cys->Met(28.031300), Delt...",A9BMX9,rpoD,acidovorans,AAASSTAAAktpakR
2,2040,AASAAAPAAGKKPAAK,"Ala->Val(28.031300), Cys->Met(28.031300), Delt...",A9BWE0,rsfS,acidovorans,AASAAAPAAGkkpaaK
3,2046,KPMVGNNVSHANNK,"Ala->Ser(15.994915), Oxidation(15.994915), Phe...",A9BNU9,rpmB,acidovorans,KPmVGNNVSHANNK
4,2148,AAASSTAAAKTPAK,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",A9BMX9,rpoD,acidovorans,AAASSTAAAktPAK
...,...,...,...,...,...,...,...
160771,52853,MLQMHSNSREDIEEAYAGDIVALAGLK,"Deoxy(-15.994915), Ser->Ala(-15.994915), Tyr->...",A0A176XDU2,fusA,tumefaciens,MLQMHSNSREDIEEAyAGDIVALAGLK
160772,52867,IHAGGFMTTVATIDVDGKK,"Ser->Val(12.036386), Thr->Xle(12.036386)",A0A176X1U1,rplY,tumefaciens,IHAGGFMTTVAtIDVDGKK
160773,52870,VTIMLEELLAAGHK,"Ala->Gly(-14.015650), Gln->Asn(-14.015650), Gl...",A0A176XJT6,A7J57_15680,tumefaciens,VTiMLEELLAAGHK
160774,53070,IHAGGFMTTVATIDVDGK,"Ser->Val(12.036386), Thr->Xle(12.036386)",A0A176X1U1,rplY,tumefaciens,IHAGGFMTTVAtIDVDGK


In [12]:
len(pd.unique(df.Organism))

19

In [13]:
df.iloc[0:2]

,scannum,Peptide,Observed Modifications,Protein ID,Gene,Organism,best_locs
0,1982,AAASSTAAAKTPAKR,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",A9BMX9,rpoD,acidovorans,AAASSTAAAkTPAKR
1,2020,AAASSTAAAKTPAKR,"Ala->Val(28.031300), Cys->Met(28.031300), Delt...",A9BMX9,rpoD,acidovorans,AAASSTAAAktpakR


In [14]:
list(df.iloc[0:2]['Observed Modifications'])

['Asn->Gln(14.015650), Asp->Glu(14.015650), Gly->Ala(14.015650), Methyl(14.015650), Ser->Thr(14.015650), Val->Xle(14.015650)',
 'Ala->Val(28.031300), Cys->Met(28.031300), Delta:H(4)C(2)(28.031300), Dimethyl(28.031300), Ethyl(28.031300)']

In [15]:
genes = df[['Gene', 'Organism']]
genes = genes.drop_duplicates()
genes = genes.groupby('Gene').count()
genes = pd.DataFrame(genes)
genes = genes[genes.Organism > 5]
genes = genes.index
homologous_genes = df[['Gene', 'Organism']]
homologous_genes = homologous_genes.drop_duplicates()
homologous_genes = homologous_genes[homologous_genes.Gene.isin(genes)]
homologous_genes.to_csv('Homologous_genes.csv')

In [16]:
len(pd.unique(homologous_genes.Gene))

280